In [ ]:
import streamlit as st
import numpy as np
import tensorflow as tf
from PIL import Image
import joblib
import requests
import tempfile


st.set_page_config(page_title="Skin Disease Classifier", layout="wide")

@st.cache_resource
def load_cnn():
    url = "https://drive.google.com/file/d/1RuGjNcW_Yi1MTjmgQymTKxP9594uzm4d/view?usp=drive_link"

    response = requests.get(url)
    temp = tempfile.NamedTemporaryFile(delete=False, suffix=".h5")
    temp.write(response.content)
    temp.flush()

    model = tf.keras.models.load_model(temp.name)
    return model

@st.cache_resource
def load_svm_ovo():
    return joblib.load("svm_ovo_model.pkl")

@st.cache_resource
def load_scaler():
    return joblib.load("feature_scaler.pkl")

cnn = load_cnn()
svm_ovo = load_svm_ovo()
scaler = load_scaler()

# Feature extractor
feature_extractor = tf.keras.Model(
    inputs=cnn.input,
    outputs=cnn.get_layer("feature_layer").output
)

class_names = ["Chickenpox", "Measles", "Monkeypox", "Normal"]



st.title("🧪 CNN + SVM Monkeypox Classifier")
st.write(
    "Aplikasi ini melakukan prediksi penyakit kulit: "
    "**Chickenpox**, **Measles**, **Monkeypox**, atau **Normal** "
    "menggunakan CNN sebagai ekstraksi fitur dan SVM sebagai classifier."
)

model_choice = st.selectbox(
    "Pilih model prediksi:",
    ["CNN Softmax", "SVM One-vs-One"]
)

uploaded_file = st.file_uploader(
    "Upload gambar kulit (jpg/png)...",
    type=["jpg", "jpeg", "png"]
)


if uploaded_file is not None:
    img = Image.open(uploaded_file).convert("RGB")
    st.image(img, caption="Gambar diunggah", width=300)

    img_resized = img.resize((224, 224))
    img_array = np.array(img_resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # CNN features
    features = feature_extractor.predict(img_array)
    features_scaled = scaler.transform(features)

    # ====== Predict Based on Model ======
    if model_choice == "CNN Softmax":
        preds = cnn.predict(img_array)
        label = class_names[np.argmax(preds)]

    elif model_choice == "SVM One-vs-One":
        pred = svm_ovo.predict(features_scaled)[0]
        label = class_names[int(pred)]

    # ====== Display Result ======
    st.success(f"### 🔍 Prediksi: **{label}**")

    st.write("---")
    st.write("Model:", model_choice)
    st.write("Confidence / raw output:")
    if model_choice == "CNN Softmax":
        st.json({class_names[i]: float(preds[0][i]) for i in range(4)})
    else:
        st.json({"SVM output": int(pred)})